## Imports

In [ ]:
#!pip install --upgrade scikit-learn

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    #!pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../titanic/"

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [4]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [5]:
 df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [6]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = None

# YOUR CODE HERE
get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [7]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [8]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}
title_dictionary

{'Capt': 'Officer',
 'Col': 'Officer',
 'Major': 'Officer',
 'Jonkheer': 'Royalty',
 'Don': 'Royalty',
 'Sir': 'Royalty',
 'Dr': 'Officer',
 'Rev': 'Officer',
 'the Countess': 'Royalty',
 'Mme': 'Mrs',
 'Mlle': 'Miss',
 'Ms': 'Mrs',
 'Mr': 'Mr',
 'Mrs': 'Mrs',
 'Miss': 'Miss',
 'Master': 'Master',
 'Lady': 'Royalty'}

In [9]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df["Title"] = df.Title.map(title_dictionary) 
df_test["Title"] = df_test.Title.map(title_dictionary)

In [10]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [11]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [13]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [14]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [36]:
# YOUR CODE HERE
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [37]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [24]:
#!pip install xgboost
#!pip install lightgbm
#!pip install catboost

In [25]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [26]:
# YOUR CODE HERE
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [27]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [28]:
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

# YOUR CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,   
    random_state=0,
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:51:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6871596	total: 144ms	remaining: 2m 23s
1:	learn: 0.6805554	total: 145ms	remaining: 1m 12s
2:	learn: 0.6746292	total: 147ms	remaining: 48.8s
3:	learn: 0.6713954	total: 147ms	remaining: 36.7s
4:	learn: 0.6657434	total: 149ms	remaining: 29.6s
5:	learn: 0.6593618	total: 150ms	remaining: 24.8s
6:	learn: 0.6536441	total: 151ms	remaining: 21.4s
7:	learn: 0.6482373	total: 153ms	remaining: 19s
8:	learn: 0.6434998	total: 155ms	remaining: 17s
9:	learn: 0.6381421	total: 156ms	remaining: 15.5s
10:	learn: 0.6331257	total: 158ms	remaining: 14.2s
11:	learn: 0.6283127	total: 159ms	remaining: 13.1s
12:	learn: 0.6253815	total: 160ms	remaining: 12.1s
13:	learn

272:	learn: 0.3539835	total: 514ms	remaining: 1.37s
273:	learn: 0.3536481	total: 515ms	remaining: 1.36s
274:	learn: 0.3535146	total: 516ms	remaining: 1.36s
275:	learn: 0.3531795	total: 518ms	remaining: 1.36s
276:	learn: 0.3530073	total: 519ms	remaining: 1.35s
277:	learn: 0.3526789	total: 521ms	remaining: 1.35s
278:	learn: 0.3525625	total: 522ms	remaining: 1.35s
279:	learn: 0.3523757	total: 523ms	remaining: 1.34s
280:	learn: 0.3520455	total: 524ms	remaining: 1.34s
281:	learn: 0.3518872	total: 526ms	remaining: 1.34s
282:	learn: 0.3517944	total: 527ms	remaining: 1.33s
283:	learn: 0.3515840	total: 528ms	remaining: 1.33s
284:	learn: 0.3513337	total: 530ms	remaining: 1.33s
285:	learn: 0.3511914	total: 532ms	remaining: 1.33s
286:	learn: 0.3510624	total: 533ms	remaining: 1.32s
287:	learn: 0.3508966	total: 535ms	remaining: 1.32s
288:	learn: 0.3507393	total: 536ms	remaining: 1.32s
289:	learn: 0.3505108	total: 537ms	remaining: 1.31s
290:	learn: 0.3503598	total: 539ms	remaining: 1.31s
291:	learn: 

527:	learn: 0.3164960	total: 853ms	remaining: 763ms
528:	learn: 0.3164612	total: 854ms	remaining: 761ms
529:	learn: 0.3162366	total: 856ms	remaining: 759ms
530:	learn: 0.3161818	total: 857ms	remaining: 757ms
531:	learn: 0.3161042	total: 858ms	remaining: 755ms
532:	learn: 0.3159990	total: 860ms	remaining: 753ms
533:	learn: 0.3158072	total: 861ms	remaining: 752ms
534:	learn: 0.3156682	total: 863ms	remaining: 750ms
535:	learn: 0.3156139	total: 864ms	remaining: 748ms
536:	learn: 0.3154684	total: 866ms	remaining: 747ms
537:	learn: 0.3153368	total: 868ms	remaining: 745ms
538:	learn: 0.3152445	total: 869ms	remaining: 743ms
539:	learn: 0.3150065	total: 870ms	remaining: 742ms
540:	learn: 0.3148915	total: 872ms	remaining: 740ms
541:	learn: 0.3148483	total: 874ms	remaining: 738ms
542:	learn: 0.3148223	total: 875ms	remaining: 736ms
543:	learn: 0.3146635	total: 876ms	remaining: 735ms
544:	learn: 0.3145178	total: 878ms	remaining: 733ms
545:	learn: 0.3144977	total: 879ms	remaining: 731ms
546:	learn: 

778:	learn: 0.2898998	total: 1.19s	remaining: 338ms
779:	learn: 0.2897708	total: 1.19s	remaining: 337ms
780:	learn: 0.2896601	total: 1.2s	remaining: 335ms
781:	learn: 0.2895581	total: 1.2s	remaining: 334ms
782:	learn: 0.2895100	total: 1.2s	remaining: 332ms
783:	learn: 0.2893728	total: 1.2s	remaining: 330ms
784:	learn: 0.2891627	total: 1.2s	remaining: 329ms
785:	learn: 0.2891068	total: 1.2s	remaining: 327ms
786:	learn: 0.2889998	total: 1.2s	remaining: 326ms
787:	learn: 0.2888020	total: 1.21s	remaining: 324ms
788:	learn: 0.2887028	total: 1.21s	remaining: 323ms
789:	learn: 0.2885982	total: 1.21s	remaining: 321ms
790:	learn: 0.2885193	total: 1.21s	remaining: 319ms
791:	learn: 0.2884367	total: 1.21s	remaining: 318ms
792:	learn: 0.2883612	total: 1.21s	remaining: 317ms
793:	learn: 0.2881392	total: 1.22s	remaining: 316ms
794:	learn: 0.2880613	total: 1.22s	remaining: 314ms
795:	learn: 0.2880167	total: 1.22s	remaining: 313ms
796:	learn: 0.2878159	total: 1.22s	remaining: 311ms
797:	learn: 0.28767

,Model,Accuracy,Bal Acc.,Time
1,Skl HistGBM,84.357542,81.870883,0.095712
2,Skl GBM,83.798883,81.146245,0.097764
3,XGBoost,81.564246,78.787879,0.093262
4,LightGBM,81.005587,78.333333,0.059839
5,CatBoost,81.005587,78.333333,1.778399
6,AdaBoost,79.888268,77.424242,0.085267
7,Random Forest,79.329609,76.969697,0.156615
8,Decision Tree,78.212291,76.060606,0.014960
9,Extra Trees,76.536313,74.156785,0.142594


In [38]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

AssertionError: 

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [30]:
# YOUR CODE HERE
from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(
    n_splits= 10,
    shuffle=True,
    random_state = 0
)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = cross_val_predict(model, x_val, y_val, cv=5)
    pred = model.predict(x_val) # PASSES ASSERT BUT WRONG!
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: Use

[16:57:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:57:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:57:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:1146: Use

[16:57:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:57:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:57:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6871596	total: 1.74ms	remain

228:	learn: 0.3631578	total: 328ms	remaining: 1.1s
229:	learn: 0.3629144	total: 330ms	remaining: 1.1s
230:	learn: 0.3627186	total: 330ms	remaining: 1.1s
231:	learn: 0.3624043	total: 332ms	remaining: 1.1s
232:	learn: 0.3620195	total: 333ms	remaining: 1.1s
233:	learn: 0.3619437	total: 334ms	remaining: 1.09s
234:	learn: 0.3615232	total: 336ms	remaining: 1.09s
235:	learn: 0.3612485	total: 337ms	remaining: 1.09s
236:	learn: 0.3610498	total: 339ms	remaining: 1.09s
237:	learn: 0.3607140	total: 341ms	remaining: 1.09s
238:	learn: 0.3606147	total: 342ms	remaining: 1.09s
239:	learn: 0.3603577	total: 344ms	remaining: 1.09s
240:	learn: 0.3600356	total: 345ms	remaining: 1.09s
241:	learn: 0.3596861	total: 349ms	remaining: 1.09s
242:	learn: 0.3593851	total: 350ms	remaining: 1.09s
243:	learn: 0.3593235	total: 352ms	remaining: 1.09s
244:	learn: 0.3591950	total: 354ms	remaining: 1.09s
245:	learn: 0.3589239	total: 355ms	remaining: 1.09s
246:	learn: 0.3585906	total: 357ms	remaining: 1.09s
247:	learn: 0.358

...

369:	learn: 0.3669930	total: 276ms	remaining: 470ms
370:	learn: 0.3669271	total: 276ms	remaining: 468ms
371:	learn: 0.3665015	total: 277ms	remaining: 468ms
372:	learn: 0.3661357	total: 278ms	remaining: 467ms
373:	learn: 0.3657960	total: 279ms	remaining: 467ms
374:	learn: 0.3654821	total: 279ms	remaining: 466ms
375:	learn: 0.3651416	total: 280ms	remaining: 465ms
376:	learn: 0.3648024	total: 281ms	remaining: 464ms
377:	learn: 0.3646335	total: 281ms	remaining: 463ms
378:	learn: 0.3643684	total: 282ms	remaining: 462ms
379:	learn: 0.3640457	total: 283ms	remaining: 461ms
380:	learn: 0.3636870	total: 283ms	remaining: 460ms
381:	learn: 0.3633102	total: 284ms	remaining: 459ms
382:	learn: 0.3630379	total: 285ms	remaining: 459ms
383:	learn: 0.3626988	total: 285ms	remaining: 458ms
384:	learn: 0.3624149	total: 286ms	remaining: 457ms
385:	learn: 0.3620879	total: 287ms	remaining: 457ms
386:	learn: 0.3618807	total: 288ms	remaining: 456ms
387:	learn: 0.3615966	total: 288ms	remaining: 455ms
388:	learn: 

587:	learn: 0.3121425	total: 441ms	remaining: 309ms
588:	learn: 0.3118690	total: 442ms	remaining: 308ms
589:	learn: 0.3116422	total: 443ms	remaining: 308ms
590:	learn: 0.3114720	total: 443ms	remaining: 307ms
591:	learn: 0.3113155	total: 444ms	remaining: 306ms
592:	learn: 0.3111808	total: 445ms	remaining: 305ms
593:	learn: 0.3108680	total: 446ms	remaining: 305ms
594:	learn: 0.3102271	total: 446ms	remaining: 304ms
595:	learn: 0.3100565	total: 447ms	remaining: 303ms
596:	learn: 0.3097759	total: 448ms	remaining: 302ms
597:	learn: 0.3096935	total: 448ms	remaining: 301ms
598:	learn: 0.3095070	total: 449ms	remaining: 300ms
599:	learn: 0.3094761	total: 449ms	remaining: 300ms
600:	learn: 0.3093430	total: 450ms	remaining: 299ms
601:	learn: 0.3092241	total: 451ms	remaining: 298ms
602:	learn: 0.3090038	total: 452ms	remaining: 298ms
603:	learn: 0.3089025	total: 453ms	remaining: 297ms
604:	learn: 0.3086962	total: 454ms	remaining: 296ms
605:	learn: 0.3084614	total: 454ms	remaining: 295ms
606:	learn: 

797:	learn: 0.2751272	total: 604ms	remaining: 153ms
798:	learn: 0.2747550	total: 605ms	remaining: 152ms
799:	learn: 0.2746547	total: 606ms	remaining: 152ms
800:	learn: 0.2745188	total: 607ms	remaining: 151ms
801:	learn: 0.2742354	total: 608ms	remaining: 150ms
802:	learn: 0.2741212	total: 608ms	remaining: 149ms
803:	learn: 0.2737635	total: 609ms	remaining: 149ms
804:	learn: 0.2737500	total: 610ms	remaining: 148ms
805:	learn: 0.2737056	total: 610ms	remaining: 147ms
806:	learn: 0.2734683	total: 611ms	remaining: 146ms
807:	learn: 0.2732103	total: 612ms	remaining: 145ms
808:	learn: 0.2730748	total: 613ms	remaining: 145ms
809:	learn: 0.2730113	total: 614ms	remaining: 144ms
810:	learn: 0.2728193	total: 614ms	remaining: 143ms
811:	learn: 0.2726607	total: 615ms	remaining: 142ms
812:	learn: 0.2724039	total: 616ms	remaining: 142ms
813:	learn: 0.2722876	total: 617ms	remaining: 141ms
814:	learn: 0.2721461	total: 618ms	remaining: 140ms
815:	learn: 0.2719997	total: 619ms	remaining: 140ms
816:	learn: 

995:	learn: 0.2471333	total: 769ms	remaining: 3.09ms
996:	learn: 0.2470689	total: 770ms	remaining: 2.32ms
997:	learn: 0.2469428	total: 771ms	remaining: 1.54ms
998:	learn: 0.2468538	total: 772ms	remaining: 773us
999:	learn: 0.2467120	total: 773ms	remaining: 0us


,Model,Accuracy,Bal Acc.,Time
1,Skl HistGBM,84.357542,81.870883,0.105749
2,Skl GBM,83.798883,81.146245,0.097283
3,XGBoost,81.564246,78.787879,0.089239
4,LightGBM,81.005587,78.333333,0.065824
5,CatBoost,81.005587,78.333333,1.637985
6,AdaBoost,79.888268,77.424242,0.087371
7,Random Forest,78.770950,75.974967,0.192795
8,Decision Tree,77.094972,75.151515,0.013929
9,Extra Trees,75.418994,72.977602,0.129758


In [31]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

AssertionError: 

# Exercise 7.1
Train with all data the best model

In [39]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]

# Fit best model with all data


# YOUR CODE HERE
best_model = tree_classifiers["Skl HistGBM"]
best_model.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [40]:
# test_pred = # Get the predictions for x_test

# YOUR CODE HERE
test_pred = best_model.predict(x_test)

In [41]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [42]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [ ]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"